# Preamble

In [16]:
from sympy import *
init_printing(use_latex='mathjax')

# Plotting
%matplotlib inline
## Make inline plots raster graphics
from IPython.display import set_matplotlib_formats
## Import modules for plotting and data analysis
import matplotlib.pyplot as plt
from matplotlib import gridspec,rc,colors
import matplotlib.ticker as plticker
## Parameters for seaborn plots
import seaborn as sns
sns.set(style='white',font_scale=1.25,
        rc={"xtick.major.size": 6, "ytick.major.size": 6,
            'text.usetex': False, 'font.family': 'serif', 'font.serif': ['Times']})

import pandas as pd
pd.set_option('mode.chained_assignment',None)

import numpy as np
from scipy.optimize import fsolve, root
from scipy.integrate import ode
backend = 'dopri5'
import warnings

# Timer
import time

import csv
from numpy.linalg import norm

from itertools import cycle
palette_size = 10;
clrs = sns.color_palette("Reds",palette_size)
iclrs = cycle(clrs) # iterated colors

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Main module

## Baseline parameter values

In [17]:
baseline = {'μ': 1/9.89244654, 'μbar': 1/12.90829551, 'd': 0.13, 'c_relative': 0.10, 'α': 0.3, 'θ': 0.45, 'κ': 40.}
tmx = 180.

## Cost of resistance

In [18]:
# Poisson brackets
PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)

In [11]:
File = open("../figures/draft/sensitivity_c.csv", 'w')
File.write("c_relative,c,outcome,sw_end_t,sw_end_x,univ_point_t,univ_point_x,err_sw_t,err_sw_x,err_univ\n")
writer = csv.writer(File,lineterminator='\n')

step = 0.01
sets = [np.arange(baseline['c_relative'],-step,-step),np.arange(baseline['c_relative']+step,0.4+step,step)]
for set_ in sets:
    end_switching_curve = {'t': 23.36, 'x': .9292}
    end_switching_curve_prev_t = end_switching_curve['t']
    tuniv = tmx-30.
    for c_relative in set_:
        # Other values of parameters
        μ = baseline['μ']; μbar = baseline['μbar']
        d = baseline['d']; α = baseline['α']; θ = baseline['θ']; κ = baseline['κ']
        # Symbolic variables
        σ, φ0, φ, x = symbols('sigma, phi0, phi, x')
        # Machinery
        A = 1-σ*(1-θ)
        Eminus = (α*A-θ)**2/2
        ΔE = A*(1-α)*((1+α)*A/2-θ)
        ΔEf = lambdify(σ,ΔE)
        b = (0.1*(exp(κ*(ΔEf(1)))+1)-0.14*(exp(κ*ΔEf(0))+1))/(exp(κ*ΔEf(1))-exp(κ*ΔEf(0))) # birth rate
        χ = 1-(0.14*(exp(κ*ΔEf(0))+1)-b*exp(κ*ΔEf(0)))/b # cost of downregulation of both pathways
        # cost of resistance (absolute value)
        c = c_relative*(b-d)/b+(1-c_relative)*χ/(exp(κ*ΔEf(0))+1)
        print("Parameters: c = %.3f (%.5f)"%(c_relative,c))
        # Hamiltonians
        h = b*(χ/(exp(κ*ΔE)+1)*(1-x)+c*x)
        H = -φ0 + φ*(b*(χ/(exp(κ*ΔE)+1)-c)*x*(1-x)+μ*(1-x)/(exp(κ*ΔE)+1)-μbar*exp(-κ*Eminus)*x) + h
        ρ = (φ*(b*χ*x+μ)+b*χ)/(exp(κ*ΔE)+1)*(1-x)-φ*μbar*exp(-κ*Eminus)*x
        # the same for sigma=0
        h0 = h.subs(σ,0); H0 = H.subs(σ,0); ρ0 = ρ.subs(σ,0)
        # Dynamics 
        ρf = lambdify((x,φ,σ),ρ)
        ρ0f = lambdify((x,φ),ρ0)
        dxdτ = lambdify((x,φ,σ),-diff(H,φ))
        dφdτ = lambdify((x,φ,σ),diff(H,x))
        dVdτ = lambdify((x,σ),h)
        dρdσ = lambdify((σ,x,φ),diff(ρ,σ))
        dδρdτ = lambdify((x,φ,σ),-PoissonBrackets(ρ0-ρ,H))
        def ode_rhs(t,state):
            x, φ, V, δρ = state
            σs = [0,1]
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            if ρf(x,φ,σstar) < ρ0f(x,φ):
                sgm = 0
            else:
                sgm = σstar
            return [dxdτ(x,φ,sgm),dφdτ(x,φ,sgm),dVdτ(x,sgm),dδρdτ(x,φ,σstar)]
        def get_δρ_ending(params):
            tme, x0 = params
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (_k<len(tms)):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(sol[0][3],(sol[1][3]-sol[0][3])/δτ)
        def get_state(tme,x0):
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (solver.t < tms[-1]) and (solver.y[0]<=1.) and (solver.y[0]>=0.):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(list(sol[0])+[(sol[1][3]-sol[0][3])/δτ])
        # Machinery for universal line
        ## Poisson brackets
        PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)
        γ0 = PoissonBrackets(PoissonBrackets(H,H0),H); γ1 = PoissonBrackets(PoissonBrackets(H0,H),H0)
        ## Dynamics
        dxdτSingExpr = -(γ0*diff(H0,φ)+γ1*diff(H,φ))/(γ0+γ1)
        dφdτSingExpr = (γ0*diff(H0,x)+γ1*diff(H,x))/(γ0+γ1)
        dVdτSingExpr = (γ0*h0+γ1*h)/(γ0+γ1)
        σSingExpr = γ1*σ/(γ0+γ1)
        ## Functional forms for the ones above
        dxdτSing = lambdify((x,φ,σ),dxdτSingExpr); dφdτSing = lambdify((x,φ,σ),dφdτSingExpr); dVdτSing = lambdify((x,φ,σ),dVdτSingExpr); σSing = lambdify((x,φ,σ),σSingExpr)
        def ode_rhs_Sing(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            return [dxdτSing(x,φ,σstar),dφdτSing(x,φ,σstar),dVdτSing(x,φ,σstar)]
        def get_state_universal(tme,end_point):
            solver = ode(ode_rhs_Sing).set_integrator(backend)
            solver.set_initial_value(end_point[1:4],end_point[0])
            solver.integrate(tme)
            return [solver.t]+list(solver.y)
        def ode_rhs_with_σstar(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σ = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σ = 1.;
            return [dxdτ(x,φ,σ),dφdτ(x,φ,σ),dVdτ(x,σ)]
        def get_state_with_σstar(tme,starting_point):
            solver = ode(ode_rhs_with_σstar).set_integrator(backend)
            solver.set_initial_value(starting_point[1:4],starting_point[0]).integrate(tme)
            return [solver.t]+list(solver.y)
        def get_finalizing_point_from_universal_curve(tme,tmx,end_point):
            unv_point = get_state_universal(tme,end_point)
            return get_state_with_σstar(tmx,unv_point)[1]
        # Get the solution!
        success = False; err = 1.
        while (not success)|(norm(err)>1e-6):
            end_switching_curve = {'t': 2*end_switching_curve['t']-end_switching_curve_prev_t-.001, 
                                   'x': end_switching_curve['x']-0.002} 
            sol = root(get_δρ_ending,(end_switching_curve['t'],end_switching_curve['x']))
            end_switching_curve_prev_t = end_switching_curve['t']
            end_switching_curve_prev_x = end_switching_curve['x']
            end_switching_curve['t'], end_switching_curve['x'] = sol.x   
            success = sol.success
            err = get_δρ_ending([end_switching_curve['t'],end_switching_curve['x']])
            if (not success): 
                print("! Trying again...", sol.message)
            elif (norm(err)>1e-6):
                print("! Trying again... Convergence is not sufficient")       
        else:
            end_point = [end_switching_curve['t']]+get_state(end_switching_curve['t'],end_switching_curve['x'])
            print("Ending point: t = %.2f, x = %.2f%%"%(end_switching_curve['t'],100*end_switching_curve['x'])," Checking the solution:",err)
            tuniv = root(get_finalizing_point_from_universal_curve,tuniv,args=(tmx,end_point)).x
            err_tuniv = get_finalizing_point_from_universal_curve(tuniv,tmx,end_point)
            univ_point = get_state_universal(tuniv,end_point)
            print("tuniv = %.2f"%tuniv," Checking the solution: ",err_tuniv)
            final_state = get_state_with_σstar(tmx,univ_point)
            outcome = exp((b-d)*tmx-final_state[-1])
            print("Fold-change in tumor size: %.2f"%(outcome))
            output = [c_relative,c,outcome,end_switching_curve['t'],end_switching_curve['x']]+list(univ_point[0:2])+list(err)+[err_tuniv]
            writer.writerow(output)
            print(output)
        
File.close()

Parameters: c = 0.100 (0.00834)
Ending point: t = 10.55, x = 82.87%  Checking the solution: (-5.8517449406102464e-08, 1.0734348617135752e-08)
tuniv = 168.86  Checking the solution:  2.5230928457631308e-12
Fold-change in tumor size: 1.22
[0.1, 0.00833519849448376, 1.21830992371149, 10.548798872306008, 0.82869008058961, 168.8567627104763, 0.6256687187408376, -5.8517449406102464e-08, 1.0734348617135752e-08, 2.5230928457631308e-12]
Parameters: c = 0.090 (0.00762)
Ending point: t = 10.55, x = 82.83%  Checking the solution: (-5.896367692653815e-08, -1.5805457911463056e-07)
tuniv = 168.94  Checking the solution:  -2.177133473502124e-12
Fold-change in tumor size: 1.23
[0.09000000000000001, 0.00762177057973159, 1.23304875540249, 10.547239442860967, 0.8283053528799607, 168.94407903527883, 0.6233828789587212, -5.896367692653815e-08, -1.5805457911463056e-07, -2.177133473502124e-12]
Parameters: c = 0.080 (0.00691)
! Trying again... The iteration is not making good progress, as measured by the 
  im

tuniv = 168.47  Checking the solution:  1.768377111410757e-14
Fold-change in tumor size: 1.12
[0.16999999999999998, 0.0133291938977490, 1.11928286542767, 10.57622286912893, 0.831343192145915, 168.4743689119304, 0.6365890004782433, -6.884160101893485e-08, 3.717173251732261e-08, 1.768377111410757e-14]
Parameters: c = 0.180 (0.01404)
Ending point: t = 10.58, x = 83.17%  Checking the solution: (-2.2060322184141605e-08, 2.6363969966461553e-09)
tuniv = 168.43  Checking the solution:  -2.4501234374696423e-14
Fold-change in tumor size: 1.11
[0.17999999999999997, 0.0140426218125011, 1.10571494188093, 10.581202639028033, 0.8317172162522871, 168.42745852928755, 0.637958008791329, -2.2060322184141605e-08, 2.6363969966461553e-09, -2.4501234374696423e-14]
Parameters: c = 0.190 (0.01476)
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Ending point: t = 10.59, x = 83.21%  Checking the solution: (-1.7144761340877258e-08, 6.401

tuniv = 167.49  Checking the solution:  4.608188830523829e-13
Fold-change in tumor size: 0.90
[0.34999999999999987, 0.0261708963632880, 0.895900700420171, 10.681600409291043, 0.8378603555971788, 167.49459731100686, 0.6624130719643054, -1.2186838407727326e-10, -1.202843395274278e-11, 4.608188830523829e-13]
Parameters: c = 0.360 (0.02688)
Ending point: t = 10.69, x = 83.82%  Checking the solution: (-6.267880516298559e-11, 6.254593077439886e-12)
tuniv = 167.44  Checking the solution:  -7.613354391367011e-14
Fold-change in tumor size: 0.88
[0.3599999999999999, 0.0268843242780402, 0.884725570337881, 10.688972406558479, 0.8382095425925722, 167.43877655441773, 0.663786468369852, -6.267880516298559e-11, 6.254593077439886e-12, -7.613354391367011e-14]
Parameters: c = 0.370 (0.02760)
Ending point: t = 10.70, x = 83.86%  Checking the solution: (7.574040677221051e-12, 1.842901212983606e-12)
tuniv = 167.38  Checking the solution:  1.1818324097134791e-13
Fold-change in tumor size: 0.87
[0.36999999999

## Alpha

In [13]:
File = open("../figures/draft/sensitivity_alpha.csv", 'w')
File.write("alpha,outcome,sw_end_t,sw_end_x,univ_point_t,univ_point_x,err_sw_t,err_sw_x,err_univ\n")
writer = csv.writer(File,lineterminator='\n')

step = 0.005
sets = [np.arange(baseline['α']+step,baseline['θ']+step,step)] #np.arange(baseline['α'],-step,-step),
for set_ in sets:
    tuniv = tmx-30.
    end_switching_curve = {'t': 21.4,'x': .84}
    end_switching_curve_prev_t = end_switching_curve['t']
    for α in set_:    
        print("Parameters: α = %.3f"%α)
        # Other values of parameters
        μ = baseline['μ']; μbar = baseline['μbar']
        d = baseline['d']; c_relative = baseline['c_relative']; θ = baseline['θ']; κ = baseline['κ']
        # Symbolic variables
        σ, φ0, φ, x = symbols('sigma, phi0, phi, x')
        # Machinery
        A = 1-σ*(1-θ)
        Eminus = (α*A-θ)**2/2
        ΔE = A*(1-α)*((1+α)*A/2-θ)
        ΔEf = lambdify(σ,ΔE)
        b = (0.1*(exp(κ*(ΔEf(1)))+1)-0.14*(exp(κ*ΔEf(0))+1))/(exp(κ*ΔEf(1))-exp(κ*ΔEf(0))) # birth rate
        χ = 1-(0.14*(exp(κ*ΔEf(0))+1)-b*exp(κ*ΔEf(0)))/b # cost of downregulation of both pathways
        # cost of resistance (absolute value)
        c = c_relative*(b-d)/b+(1-c_relative)*χ/(exp(κ*ΔEf(0))+1)
        # Hamiltonians
        h = b*(χ/(exp(κ*ΔE)+1)*(1-x)+c*x)
        H = -φ0 + φ*(b*(χ/(exp(κ*ΔE)+1)-c)*x*(1-x)+μ*(1-x)/(exp(κ*ΔE)+1)-μbar*exp(-κ*Eminus)*x) + h
        ρ = (φ*(b*χ*x+μ)+b*χ)/(exp(κ*ΔE)+1)*(1-x)-φ*μbar*exp(-κ*Eminus)*x
        # the same for sigma=0
        h0 = h.subs(σ,0); H0 = H.subs(σ,0); ρ0 = ρ.subs(σ,0)
        # Dynamics 
        ρf = lambdify((x,φ,σ),ρ)
        ρ0f = lambdify((x,φ),ρ0)
        dxdτ = lambdify((x,φ,σ),-diff(H,φ))
        dφdτ = lambdify((x,φ,σ),diff(H,x))
        dVdτ = lambdify((x,σ),h)
        dρdσ = lambdify((σ,x,φ),diff(ρ,σ))
        dδρdτ = lambdify((x,φ,σ),-PoissonBrackets(ρ0-ρ,H))
        def ode_rhs(t,state):
            x, φ, V, δρ = state
            σs = [0,1]
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            if ρf(x,φ,σstar) < ρ0f(x,φ):
                sgm = 0
            else:
                sgm = σstar
            return [dxdτ(x,φ,sgm),dφdτ(x,φ,sgm),dVdτ(x,sgm),dδρdτ(x,φ,σstar)]
        def get_δρ_ending(params):
            tme, x0 = params
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-9; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (_k<len(tms)):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(sol[0][3],(sol[1][3]-sol[0][3])/δτ)
        def get_state(tme,x0):
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (solver.t < tms[-1]) and (solver.y[0]<=1.) and (solver.y[0]>=0.):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(list(sol[0])+[(sol[1][3]-sol[0][3])/δτ])
        # Machinery for universal line
        ## Poisson brackets
        PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)
        γ0 = PoissonBrackets(PoissonBrackets(H,H0),H); γ1 = PoissonBrackets(PoissonBrackets(H0,H),H0)
        ## Dynamics
        dxdτSingExpr = -(γ0*diff(H0,φ)+γ1*diff(H,φ))/(γ0+γ1)
        dφdτSingExpr = (γ0*diff(H0,x)+γ1*diff(H,x))/(γ0+γ1)
        dVdτSingExpr = (γ0*h0+γ1*h)/(γ0+γ1)
        σSingExpr = γ1*σ/(γ0+γ1)
        ## Functional forms for the ones above
        dxdτSing = lambdify((x,φ,σ),dxdτSingExpr); dφdτSing = lambdify((x,φ,σ),dφdτSingExpr); dVdτSing = lambdify((x,φ,σ),dVdτSingExpr); σSing = lambdify((x,φ,σ),σSingExpr)
        def ode_rhs_Sing(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            return [dxdτSing(x,φ,σstar),dφdτSing(x,φ,σstar),dVdτSing(x,φ,σstar)]
        def get_state_universal(tme,end_point):
            solver = ode(ode_rhs_Sing).set_integrator(backend)
            solver.set_initial_value(end_point[1:4],end_point[0])
            solver.integrate(tme)
            return [solver.t]+list(solver.y)
        def ode_rhs_with_σstar(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σ = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σ = 1.;
            return [dxdτ(x,φ,σ),dφdτ(x,φ,σ),dVdτ(x,σ)]
        def get_state_with_σstar(tme,starting_point):
            solver = ode(ode_rhs_with_σstar).set_integrator(backend)
            solver.set_initial_value(starting_point[1:4],starting_point[0]).integrate(tme)
            return [solver.t]+list(solver.y)
        def get_finalizing_point_from_universal_curve(tme,tmx,end_point):
            unv_point = get_state_universal(tme,end_point)
            return get_state_with_σstar(tmx,unv_point)[1]
        # Get the solution!
        success = False; err = 1.
        while (not success)|(norm(err)>1e-5):
            end_switching_curve = {'t': 2*end_switching_curve['t']-end_switching_curve_prev_t-.002, 
                                   'x': end_switching_curve['x']+0.01} 
            sol = root(get_δρ_ending,(end_switching_curve['t'],end_switching_curve['x']))
            end_switching_curve_prev_t = end_switching_curve['t']
            end_switching_curve_prev_x = end_switching_curve['x']
            end_switching_curve['t'], end_switching_curve['x'] = sol.x   
            success = sol.success
            err = get_δρ_ending([end_switching_curve['t'],end_switching_curve['x']])
            if (not success): 
                print("! Trying again...", sol.message)
            elif (norm(err)>1e-6):
                print("! Trying again... Convergence is not sufficient")       
        else:
            end_point = [end_switching_curve['t']]+get_state(end_switching_curve['t'],end_switching_curve['x'])
            print("Ending point: t = %.2f, x = %.2f%%"%(end_switching_curve['t'],100*end_switching_curve['x'])," Checking the solution:",err)
            tuniv = root(get_finalizing_point_from_universal_curve,tuniv,args=(tmx,2*end_point)).x
            err_tuniv = get_finalizing_point_from_universal_curve(tuniv,tmx,end_point)
            univ_point = get_state_universal(tuniv,end_point)
            print("tuniv = %.2f"%tuniv," Checking the solution: ",err_tuniv)
            final_state = get_state_with_σstar(tmx,univ_point)
            outcome = exp((b-d)*tmx-final_state[-1])
            print("Fold-change in tumor size: %.2f"%(outcome))
            output = [α,outcome,end_switching_curve['t'],end_switching_curve['x']]+list(univ_point[0:2])+list(err)+[err_tuniv]
            writer.writerow(output)
            print(output)
        
File.close()

Parameters: α = 0.305
Ending point: t = 10.48, x = 82.50%  Checking the solution: (-7.89733596799066e-11, -3.196349963996782e-11)
tuniv = 168.95  Checking the solution:  1.207506317157936e-13
Fold-change in tumor size: 1.19
[0.305, 1.18933847083105, 10.47690540700752, 0.8250431197139906, 168.9497860102835, 0.6210881350138265, -7.89733596799066e-11, -3.196349963996782e-11, 1.207506317157936e-13]
Parameters: α = 0.310
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Ending point: t = 10.41, x = 82.14%  Checking t

Ending point: t = 10.25, x = 77.36%  Checking the solution: (1.0942659133180712e-11, 3.295199985901429e-12)
tuniv = 169.35  Checking the solution:  -2.1614654510671016e-13
Fold-change in tumor size: 0.84
[0.38500000000000006, 0.842027838649093, 10.24983505222961, 0.773578277734634, 169.35066356537484, 0.5704804479296421, 1.0942659133180712e-11, 3.295199985901429e-12, -2.1614654510671016e-13]
Parameters: α = 0.390
! Trying again... Convergence is not sufficient
Ending point: t = 10.35, x = 77.09%  Checking the solution: (-1.8017525744610204e-07, -4.983259417786612e-06)
tuniv = 170.13  Checking the solution:  -4.4058506842858947e-13
Fold-change in tumor size: 0.83
[0.39000000000000007, 0.827373434174153, 10.348568744008118, 0.7708733636591372, 170.13098234729347, 0.5520802842400259, -1.8017525744610204e-07, -4.983259417786612e-06, -4.4058506842858947e-13]
Parameters: α = 0.395
! Trying again... Convergence is not sufficient
Ending point: t = 10.39, x = 76.82%  Checking the solution: (-1.

## Theta

In [27]:
File = open("../figures/draft/sensitivity_theta.csv", 'a')
# File.write("theta,outcome,sw_end_t,sw_end_x,univ_point_t,univ_point_x,err_sw_t,err_sw_x,err_univ\n")
writer = csv.writer(File,lineterminator='\n')

step = 0.005
sets = [np.arange(baseline['θ']+step,1.0+step,step)] #,np.arange(baseline['θ'],baseline['α']-step,-step)
for set_ in sets:
    end_switching_curve = {'t': 30.2, 'x': .8874}
    tuniv = tmx-30.
    end_switching_curve_prev_t = end_switching_curve['t']
    for θ in set_:    
        print("Parameters: θ = %.4f"%θ)
        # Other values of parameters
        μ = baseline['μ']; μbar = baseline['μbar']
        d = baseline['d']; c_relative = baseline['c_relative']; α = baseline['α']; κ = baseline['κ']
        # Symbolic variables
        σ, φ0, φ, x = symbols('sigma, phi0, phi, x')
        # Machinery
        A = 1-σ*(1-θ)
        Eminus = (α*A-θ)**2/2
        ΔE = A*(1-α)*((1+α)*A/2-θ)
        ΔEf = lambdify(σ,ΔE)
        b = (0.1*(exp(κ*(ΔEf(1)))+1)-0.14*(exp(κ*ΔEf(0))+1))/(exp(κ*ΔEf(1))-exp(κ*ΔEf(0))) # birth rate
        χ = 1-(0.14*(exp(κ*ΔEf(0))+1)-b*exp(κ*ΔEf(0)))/b # cost of downregulation of both pathways
        # cost of resistance (absolute value)
        c = c_relative*(b-d)/b+(1-c_relative)*χ/(exp(κ*ΔEf(0))+1)
        # Hamiltonians
        h = b*(χ/(exp(κ*ΔE)+1)*(1-x)+c*x)
        H = -φ0 + φ*(b*(χ/(exp(κ*ΔE)+1)-c)*x*(1-x)+μ*(1-x)/(exp(κ*ΔE)+1)-μbar*exp(-κ*Eminus)*x) + h
        ρ = (φ*(b*χ*x+μ)+b*χ)/(exp(κ*ΔE)+1)*(1-x)-φ*μbar*exp(-κ*Eminus)*x
        # the same for sigma=0
        h0 = h.subs(σ,0); H0 = H.subs(σ,0); ρ0 = ρ.subs(σ,0)
        # Dynamics 
        ρf = lambdify((x,φ,σ),ρ)
        ρ0f = lambdify((x,φ),ρ0)
        dxdτ = lambdify((x,φ,σ),-diff(H,φ))
        dφdτ = lambdify((x,φ,σ),diff(H,x))
        dVdτ = lambdify((x,σ),h)
        dρdσ = lambdify((σ,x,φ),diff(ρ,σ))
        dδρdτ = lambdify((x,φ,σ),-PoissonBrackets(ρ0-ρ,H))
        def ode_rhs(t,state):
            x, φ, V, δρ = state
            σs = [0,1]
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            if ρf(x,φ,σstar) < ρ0f(x,φ):
                sgm = 0
            else:
                sgm = σstar
            return [dxdτ(x,φ,sgm),dφdτ(x,φ,sgm),dVdτ(x,sgm),dδρdτ(x,φ,σstar)]
        def get_δρ_ending(params):
            tme, x0 = params
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-9; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (_k<len(tms)):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(sol[0][3],(sol[1][3]-sol[0][3])/δτ)
        def get_state(tme,x0):
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (solver.t < tms[-1]) and (solver.y[0]<=1.) and (solver.y[0]>=0.):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(list(sol[0])+[(sol[1][3]-sol[0][3])/δτ])
        # Machinery for universal line
        ## Poisson brackets
        PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)
        γ0 = PoissonBrackets(PoissonBrackets(H,H0),H); γ1 = PoissonBrackets(PoissonBrackets(H0,H),H0)
        ## Dynamics
        dxdτSingExpr = -(γ0*diff(H0,φ)+γ1*diff(H,φ))/(γ0+γ1)
        dφdτSingExpr = (γ0*diff(H0,x)+γ1*diff(H,x))/(γ0+γ1)
        dVdτSingExpr = (γ0*h0+γ1*h)/(γ0+γ1)
        σSingExpr = γ1*σ/(γ0+γ1)
        ## Functional forms for the ones above
        dxdτSing = lambdify((x,φ,σ),dxdτSingExpr); dφdτSing = lambdify((x,φ,σ),dφdτSingExpr); dVdτSing = lambdify((x,φ,σ),dVdτSingExpr); σSing = lambdify((x,φ,σ),σSingExpr)
        def ode_rhs_Sing(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            return [dxdτSing(x,φ,σstar),dφdτSing(x,φ,σstar),dVdτSing(x,φ,σstar)]
        def get_state_universal(tme,end_point):
            solver = ode(ode_rhs_Sing).set_integrator(backend)
            solver.set_initial_value(end_point[1:4],end_point[0])
            solver.integrate(tme)
            return [solver.t]+list(solver.y)
        def ode_rhs_with_σstar(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σ = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σ = 1.;
            return [dxdτ(x,φ,σ),dφdτ(x,φ,σ),dVdτ(x,σ)]
        def get_state_with_σstar(tme,starting_point):
            solver = ode(ode_rhs_with_σstar).set_integrator(backend)
            solver.set_initial_value(starting_point[1:4],starting_point[0]).integrate(tme)
            return [solver.t]+list(solver.y)
        def get_finalizing_point_from_universal_curve(tme,tmx,end_point):
            unv_point = get_state_universal(tme,end_point)
            return get_state_with_σstar(tmx,unv_point)[1]
        # Get the solution!
        success = False; err = 1.
        while (not success)|(norm(err)>1e-4):
            end_switching_curve = {'t': 2*end_switching_curve['t']-end_switching_curve_prev_t-.001, 
                                   'x': end_switching_curve['x']*0.95} 
            sol = root(get_δρ_ending,(end_switching_curve['t'],end_switching_curve['x']))
            end_switching_curve_prev_t = end_switching_curve['t']
            end_switching_curve_prev_x = end_switching_curve['x']
            end_switching_curve['t'], end_switching_curve['x'] = sol.x   
            success = sol.success
            err = get_δρ_ending([end_switching_curve['t'],end_switching_curve['x']])
            if (not success): 
                print("! Trying again...", sol.message)
            elif (norm(err)>1e-5):
                print("! Trying again... Convergence is not sufficient")       
        else:
            end_point = [end_switching_curve['t']]+get_state(end_switching_curve['t'],end_switching_curve['x'])
            print("Ending point: t = %.2f, x = %.2f%%"%(end_switching_curve['t'],100*end_switching_curve['x'])," Checking the solution:",err)
            tuniv = root(get_finalizing_point_from_universal_curve,tuniv,args=(tmx,end_point)).x
            err_tuniv = get_finalizing_point_from_universal_curve(tuniv,tmx,end_point)
            univ_point = get_state_universal(tuniv,end_point)
            print("tuniv = %.2f"%tuniv," Checking the solution: ",err_tuniv)
            final_state = get_state_with_σstar(tmx,univ_point)
            outcome = exp((b-d)*tmx-final_state[-1])
            print("Fold-change in tumor size: %.2f"%(outcome))
            output = [θ,outcome,end_switching_curve['t'],end_switching_curve['x']]+list(univ_point[0:2])+list(err)+[err_tuniv]
            writer.writerow(output)
            print(output)
        
File.close()

Parameters: θ = 0.4550
Ending point: t = 10.60, x = 83.28%  Checking the solution: (6.286715731719186e-11, 1.6625968420797864e-11)
tuniv = 168.78  Checking the solution:  8.755930008819135e-14
Fold-change in tumor size: 1.25
[0.455, 1.25274312002472, 10.59561774416366, 0.8328474497279053, 168.782172215961, 0.6303448189877247, 6.286715731719186e-11, 1.6625968420797864e-11, 8.755930008819135e-14]
Parameters: θ = 0.4600
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
Ending point: t = 10.65, x = 83.70%  Checking the solution: (-4.7197355342123794e-08, -8.174284840032725e-09)
tuniv = 168.70  Checking the solution:  -5.875439024194407e-13
Fold-change in tumor size: 1.29
[0.46, 1.28813040657978, 10.650451494494792, 0.8370070248459077, 168.69535583854673, 0.6352377307787797, -4.7197355342123794e-08, -8.174284840032725e-09, -5.875439024194407e-13]
Parameters: θ = 0.4650
! Trying again... The iteration is no

Ending point: t = 12.16, x = 89.54%  Checking the solution: (-4.6835091485872334e-08, -1.000724163857837e-07)
tuniv = 165.80  Checking the solution:  -2.6285917886781363e-13
Fold-change in tumor size: 1.89
[0.53, 1.89003972526072, 12.163246244242478, 0.8953739473828703, 165.8032449683101, 0.732016675346453, -4.6835091485872334e-08, -1.000724163857837e-07, -2.6285917886781363e-13]
Parameters: θ = 0.5350
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Ending point: t = 12.32, x = 89.95%  Checking the solution: (-8.466975560885055e-13, -1.9976129065407187e-13)
tuniv = 165.35  Checking the solution:  5.166700400849322e-14
Fold-change in tumor size: 1.94
[0.5350000000000001, 1.94092261214637, 12.319085754668032, 0.8995013062350735, 165.35310676776368, 0.7423065892635194, -8.466975560885055e-13, -1.9976129065407187e-13, 5.166700400849322e-14]
Parameters: θ = 0.5400
! Trying again... Convergence is not sufficient
! T

KeyboardInterrupt: 

## kappa

In [37]:
File = open("../figures/draft/sensitivity_kappa.csv", 'a')
# File.write("kappa,outcome,sw_end_t,sw_end_x,univ_point_t,univ_point_x,err_sw_t,err_sw_x,err_univ\n")
writer = csv.writer(File,lineterminator='\n')

step = 2
# sets = [np.arange(120,100+step,step),np.arange(17.5,-step,-step)]
sets = [np.arange(baseline['κ'],80+step,step)] #,np.arange(baseline['κ'],-step,-step)
# sets = [np.arange(24,-step,-step)]
# sets = [np.arange(.199,-step,-step)]
for set_ in sets:
#     end_switching_curve = {'t': 30.2, 'x': .8874}
    end_switching_curve = {'t': 61.91, 'x': .7502}
    tuniv = tmx-20.
    end_switching_curve_prev_t = end_switching_curve['t']
    for κ in set_:    
        print("Parameters: κ = %.4f"%κ)
        # Other values of parameters
        μ = baseline['μ']; μbar = baseline['μbar']
        d = baseline['d']; c_relative = baseline['c_relative']; α = baseline['α']; θ = baseline['θ']
        # Symbolic variables
        σ, φ0, φ, x = symbols('sigma, phi0, phi, x')
        # Machinery
        A = 1-σ*(1-θ)
        Eminus = (α*A-θ)**2/2
        ΔE = A*(1-α)*((1+α)*A/2-θ)
        ΔEf = lambdify(σ,ΔE)
        b = (0.1*(exp(κ*(ΔEf(1)))+1)-0.14*(exp(κ*ΔEf(0))+1))/(exp(κ*ΔEf(1))-exp(κ*ΔEf(0))) # birth rate
        χ = 1-(0.14*(exp(κ*ΔEf(0))+1)-b*exp(κ*ΔEf(0)))/b # cost of downregulation of both pathways
        # cost of resistance (absolute value)
        c = c_relative*(b-d)/b+(1-c_relative)*χ/(exp(κ*ΔEf(0))+1)
        # Hamiltonians
        h = b*(χ/(exp(κ*ΔE)+1)*(1-x)+c*x)
        H = -φ0 + φ*(b*(χ/(exp(κ*ΔE)+1)-c)*x*(1-x)+μ*(1-x)/(exp(κ*ΔE)+1)-μbar*exp(-κ*Eminus)*x) + h
        ρ = (φ*(b*χ*x+μ)+b*χ)/(exp(κ*ΔE)+1)*(1-x)-φ*μbar*exp(-κ*Eminus)*x
        # the same for sigma=0
        h0 = h.subs(σ,0); H0 = H.subs(σ,0); ρ0 = ρ.subs(σ,0)
        # Dynamics 
        ρf = lambdify((x,φ,σ),ρ)
        ρ0f = lambdify((x,φ),ρ0)
        dxdτ = lambdify((x,φ,σ),-diff(H,φ))
        dφdτ = lambdify((x,φ,σ),diff(H,x))
        dVdτ = lambdify((x,σ),h)
        dρdσ = lambdify((σ,x,φ),diff(ρ,σ))
        dδρdτ = lambdify((x,φ,σ),-PoissonBrackets(ρ0-ρ,H))
        def ode_rhs(t,state):
            x, φ, V, δρ = state
            σs = [0,1]
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            if ρf(x,φ,σstar) < ρ0f(x,φ):
                sgm = 0
            else:
                sgm = σstar
            return [dxdτ(x,φ,sgm),dφdτ(x,φ,sgm),dVdτ(x,sgm),dδρdτ(x,φ,σstar)]
        def get_δρ_ending(params):
            tme, x0 = params
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-9; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (_k<len(tms)):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(sol[0][3],(sol[1][3]-sol[0][3])/δτ)
        def get_state(tme,x0):
            solver = ode(ode_rhs).set_integrator(backend)
            δρ0 = ρ0.subs(x,x0).subs(φ,0)-ρ.subs(x,x0).subs(φ,0).subs(σ,1.)
            solver.set_initial_value([x0,0,0,δρ0],0.)
            δτ = 1.0e-8; tms = [tme,tme+δτ]
            _k = 0; sol = []
            while (solver.t < tms[-1]) and (solver.y[0]<=1.) and (solver.y[0]>=0.):
                solver.integrate(tms[_k])
                sol.append(solver.y)
                _k += 1
            return(list(sol[0])+[(sol[1][3]-sol[0][3])/δτ])
        # Machinery for universal line
        ## Poisson brackets
        PoissonBrackets = lambda H1, H2: diff(H1,x)*diff(H2,φ)-diff(H1,φ)*diff(H2,x)
        γ0 = PoissonBrackets(PoissonBrackets(H,H0),H); γ1 = PoissonBrackets(PoissonBrackets(H0,H),H0)
        ## Dynamics
        dxdτSingExpr = -(γ0*diff(H0,φ)+γ1*diff(H,φ))/(γ0+γ1)
        dφdτSingExpr = (γ0*diff(H0,x)+γ1*diff(H,x))/(γ0+γ1)
        dVdτSingExpr = (γ0*h0+γ1*h)/(γ0+γ1)
        σSingExpr = γ1*σ/(γ0+γ1)
        ## Functional forms for the ones above
        dxdτSing = lambdify((x,φ,σ),dxdτSingExpr); dφdτSing = lambdify((x,φ,σ),dφdτSingExpr); dVdτSing = lambdify((x,φ,σ),dVdτSingExpr); σSing = lambdify((x,φ,σ),σSingExpr)
        def ode_rhs_Sing(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σstar = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σstar = 1.;
            return [dxdτSing(x,φ,σstar),dφdτSing(x,φ,σstar),dVdτSing(x,φ,σstar)]
        def get_state_universal(tme,end_point):
            solver = ode(ode_rhs_Sing).set_integrator(backend)
            solver.set_initial_value(end_point[1:4],end_point[0])
            solver.integrate(tme)
            return [solver.t]+list(solver.y)
        def ode_rhs_with_σstar(t,state):
            x, φ, V = state
            if (dρdσ(1.,x,φ)<0) and (dρdσ(θ,x,φ)>0):
                σ = fsolve(dρdσ,.8,args=(x,φ,))[0]
            else:
                σ = 1.;
            return [dxdτ(x,φ,σ),dφdτ(x,φ,σ),dVdτ(x,σ)]
        def get_state_with_σstar(tme,starting_point):
            solver = ode(ode_rhs_with_σstar).set_integrator(backend)
            solver.set_initial_value(starting_point[1:4],starting_point[0]).integrate(tme)
            return [solver.t]+list(solver.y)
        def get_finalizing_point_from_universal_curve(tme,tmx,end_point):
            unv_point = get_state_universal(tme,end_point)
            return get_state_with_σstar(tmx,unv_point)[1]
        # Get the solution!
        success = False; err = 1.
        while (not success)|(norm(err)>1e-4):
            end_switching_curve = {'t': 2*end_switching_curve['t']-.01-end_switching_curve_prev_t,
                                   'x': end_switching_curve['x']*1.02} 
            sol = root(get_δρ_ending,(end_switching_curve['t'],end_switching_curve['x']))
            end_switching_curve_prev_t = end_switching_curve['t']
            end_switching_curve_prev_x = end_switching_curve['x']
            end_switching_curve['t'], end_switching_curve['x'] = sol.x   
            success = sol.success
            err = get_δρ_ending([end_switching_curve['t'],end_switching_curve['x']])
            if (not success): 
                print("! Trying again...", sol.message)
            elif (norm(err)>1e-4):
                print("! Trying again... Convergence is not sufficient")       
        else:
            end_point = [end_switching_curve['t']]+get_state(end_switching_curve['t'],end_switching_curve['x'])
            print("Ending point: t = %.2f, x = %.2f%%"%(end_switching_curve['t'],100*end_switching_curve['x'])," Checking the solution:",err)
            tuniv = root(get_finalizing_point_from_universal_curve,tuniv,args=(tmx,end_point)).x
            err_tuniv = get_finalizing_point_from_universal_curve(tuniv,tmx,end_point)
            univ_point = get_state_universal(tuniv,end_point)
            print("tuniv = %.2f"%tuniv," Checking the solution: ",err_tuniv)
            final_state = get_state_with_σstar(tmx,univ_point)
            outcome = exp((b-d)*tmx-final_state[-1])
            print("Fold-change in tumor size: %.2f"%(outcome))
            output = [κ,outcome,end_switching_curve['t'],end_switching_curve['x']]+list(univ_point[0:2])+list(err)+[err_tuniv]
            writer.writerow(output)
            print(output)
        
File.close()

Parameters: κ = 40.0000
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
! Trying again... Convergence is not sufficient
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
! Trying again... Convergence is not sufficient
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
Ending point: t = 10.55, x = 82.87%  Checking the solution: (-5.6698653323331914e-08, -3.160487846324566e-07)
tuniv = 168.92  Checking the solution:  1.5187850976872141e-13
Fold-change in tumor size: 1.22
[40.0, 1.21831533374661, 10.55236026322317, 0.8286901096457623, 168.92225439511293, 0.6243056158484195, -5.6698653323331914e-08, -3.160487846324566e-07, 

Ending point: t = 9.36, x = 86.69%  Checking the solution: (-1.208427045359733e-12, -3.5288058075266345e-13)
tuniv = 170.00  Checking the solution:  9.39352762241441e-14
Fold-change in tumor size: 1.59
[60.0, 1.59383999195095, 9.362475905496114, 0.866889002377346, 170.00196013485086, 0.6451905205161177, -1.208427045359733e-12, -3.5288058075266345e-13, 9.39352762241441e-14]
Parameters: κ = 62.0000
! Trying again... Convergence is not sufficient
! Trying again... The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
Ending point: t = 9.33, x = 86.97%  Checking the solution: (-4.984697544629295e-12, -3.0410543972494802e-12)
tuniv = 170.03  Checking the solution:  1.0044742815296104e-13
Fold-change in tumor size: 1.63
[62.0, 1.62503064296849, 9.326633687502204, 0.8697122313269088, 170.02572607668716, 0.6476208714021148, -4.984697544629295e-12, -3.0410543972494802e-12, 1.0044742815296104e-13]
Parameters: κ = 64.0000
Ending poin

In [38]:
File.close()